In [37]:
import numpy as np
import gc

In [38]:
a = []
b = [1,2,3]
c = [4,5,6]

a.append(b)
a.append(c)

del b
del c
gc.collect()



print(a)
print(b)
print(c)

[[1, 2, 3], [4, 5, 6]]


NameError: name 'b' is not defined

In [26]:
# create an empty 3D array with 2 layers, 3 rows and 4 columns
arr = np.empty((2, 3, 4))

# print the array
print(arr)

[[[4.6430201e-310 0.0000000e+000 0.0000000e+000 0.0000000e+000]
  [0.0000000e+000 0.0000000e+000 0.0000000e+000 0.0000000e+000]
  [0.0000000e+000 0.0000000e+000 0.0000000e+000 0.0000000e+000]]

 [[0.0000000e+000 0.0000000e+000 0.0000000e+000 0.0000000e+000]
  [0.0000000e+000 0.0000000e+000 0.0000000e+000 0.0000000e+000]
  [0.0000000e+000 0.0000000e+000 0.0000000e+000 0.0000000e+000]]]


In [23]:
#na = np.array([[1, 2, 3]], dtype=np.float64)
na = np.array()

#na = np.append(na, [[4, 5, 6]], axis=0)


na

array([[1., 2., 3.],
       [4., 5., 6.]])

# LSTM Issue

In [ ]:
!pip install tensorflow_datasets

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import os
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.client import device_lib

dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True,
                          as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

BUFFER_SIZE = 10000
BATCH_SIZE = 64

train_dataset = train_dataset.shuffle(BUFFER_SIZE)
train_dataset = train_dataset.padded_batch(BATCH_SIZE, train_dataset.output_shapes)

test_dataset = test_dataset.padded_batch(BATCH_SIZE, test_dataset.output_shapes)
encoder = info.features['text'].encoder


model = tf.keras.Sequential([
    tf.keras.layers.Embedding(encoder.vocab_size, 64),
    #tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])
history = model.fit(train_dataset, epochs=10,
                    validation_data=test_dataset, 
                    validation_steps=30)

ModuleNotFoundError: No module named 'tensorflow_datasets'